<a href="https://colab.research.google.com/github/lewisnjue/kaggle_competition_house_price/blob/main/impored_version_of_house_price_prediction_competion_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
from google.colab import files
files.upload()

{}

In [52]:

!mv kaggle.json /root/.config/kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory


In [53]:
!ls


data_description.txt				 sample_data		test.csv
house-prices-advanced-regression-techniques.zip  sample_submission.csv	train.csv


In [54]:
!kaggle competitions download -c house-prices-advanced-regression-techniques


house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [55]:
!unzip house-prices-advanced-regression-techniques

Archive:  house-prices-advanced-regression-techniques.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [81]:
!ls


data_description.txt				 sample_data		test.csv
house-prices-advanced-regression-techniques.zip  sample_submission.csv	train.csv


In [82]:
columns = {
    "target": ["SalePrice"],  # Target variable
    "general_info": [
        "MSSubClass", "MSZoning", "Neighborhood", "Condition1", "Condition2",
        "BldgType", "HouseStyle", "Functional", "SaleType", "SaleCondition"
    ],
    "lot_info": [
        "LotFrontage", "LotArea", "LotShape", "LandContour", "Utilities",
        "LotConfig", "LandSlope", "Street", "Alley"
    ],
    "building_info": [
        "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", "RoofStyle",
        "RoofMatl", "Exterior1st", "Exterior2nd", "ExterQual", "ExterCond",
        "Foundation", "Heating", "HeatingQC", "CentralAir", "Electrical"
    ],
    "basement_info": [
        "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinSF1",
        "BsmtFinType2", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF",
        "BsmtFullBath", "BsmtHalfBath"
    ],
    "living_area_info": [
        "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",
        "TotRmsAbvGrd", "Bedroom", "Kitchen", "KitchenQual"
    ],
    "bathroom_info": [
        "FullBath", "HalfBath"
    ],
    "fireplace_info": [
        "Fireplaces", "FireplaceQu"
    ],
    "garage_info": [
        "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars",
        "GarageArea", "GarageQual", "GarageCond", "PavedDrive"
    ],
    "porch_and_deck_info": [
        "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch",
        "ScreenPorch"
    ],
    "pool_and_fence_info": [
        "PoolArea", "PoolQC", "Fence", "MiscFeature", "MiscVal"
    ],
    "time_info": [
        "MoSold", "YrSold"
    ]
}


In [83]:
feature_columns = (
    columns["general_info"] +
    columns["lot_info"] +
    columns["building_info"] +
    columns["living_area_info"] +
    columns["garage_info"]
)



In [84]:
import matplotlib
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [85]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # device agnostic code
device

'cuda'

In [86]:
df = pd.read_csv('train.csv',index_col='Id')
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [87]:
target = df[columns['target']]
target.head()

,SalePrice
Id,
1,208500
2,181500
3,223500
4,140000
5,250000


In [88]:
target.isna().sum() # no column without sale price

,0
SalePrice,0


In [89]:
data_predictors = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF',
                        'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
labels = df[data_predictors]
labels.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
1,8450,2003,856,854,2,3,8
2,9600,1976,1262,0,2,3,6
3,11250,2001,920,866,2,3,6
4,9550,1915,961,756,1,3,7
5,14260,2000,1145,1053,2,4,9


In [90]:
labels.isna().sum()

,0
LotArea,0
YearBuilt,0
1stFlrSF,0
2ndFlrSF,0
FullBath,0
BedroomAbvGr,0
TotRmsAbvGrd,0


In [91]:
labels.select_dtypes(include='number').columns

Index(['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath',
       'BedroomAbvGr', 'TotRmsAbvGrd'],
      dtype='object')

In [92]:
labels.shape

(1460, 7)

In [93]:
target.shape

(1460, 1)

In [94]:
labels

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
1,8450,2003,856,854,2,3,8
2,9600,1976,1262,0,2,3,6
3,11250,2001,920,866,2,3,6
4,9550,1915,961,756,1,3,7
5,14260,2000,1145,1053,2,4,9
...,...,...,...,...,...,...,...
1456,7917,1999,953,694,2,3,7
1457,13175,1978,2073,0,2,3,7
1458,9042,1941,1188,1152,2,4,9


In [95]:
# convert my lables into tensors
labels = torch.from_numpy(labels.values).type(torch.float32)
labels

tensor([[8.4500e+03, 2.0030e+03, 8.5600e+02,  ..., 2.0000e+00, 3.0000e+00,
         8.0000e+00],
        [9.6000e+03, 1.9760e+03, 1.2620e+03,  ..., 2.0000e+00, 3.0000e+00,
         6.0000e+00],
        [1.1250e+04, 2.0010e+03, 9.2000e+02,  ..., 2.0000e+00, 3.0000e+00,
         6.0000e+00],
        ...,
        [9.0420e+03, 1.9410e+03, 1.1880e+03,  ..., 2.0000e+00, 4.0000e+00,
         9.0000e+00],
        [9.7170e+03, 1.9500e+03, 1.0780e+03,  ..., 1.0000e+00, 2.0000e+00,
         5.0000e+00],
        [9.9370e+03, 1.9650e+03, 1.2560e+03,  ..., 1.0000e+00, 3.0000e+00,
         6.0000e+00]])

In [96]:
labels.shape

torch.Size([1460, 7])

In [97]:
# convert my target to tensor
target = torch.from_numpy(target.values).type(torch.float32)
target

tensor([[208500.],
        [181500.],
        [223500.],
        ...,
        [266500.],
        [142125.],
        [147500.]])

In [115]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
labels = torch.tensor(scaler.fit_transform(labels.numpy()), dtype=torch.float32)
target = torch.tensor(scaler.fit_transform(target.numpy()), dtype=torch.float32)
labels


tensor([[-0.2071,  1.0510, -0.7934,  ...,  0.7897,  0.1638,  0.9122],
        [-0.0919,  0.1567,  0.2571,  ...,  0.7897,  0.1638, -0.3187],
        [ 0.0735,  0.9848, -0.6278,  ...,  0.7897,  0.1638, -0.3187],
        ...,
        [-0.1478, -1.0025,  0.0657,  ...,  0.7897,  1.3900,  1.5277],
        [-0.0802, -0.7044, -0.2190,  ..., -1.0260, -1.0625, -0.9341],
        [-0.0581, -0.2076,  0.2416,  ..., -1.0260,  0.1638, -0.3187]])

In [116]:
target

tensor([[ 0.3473],
        [ 0.0073],
        [ 0.5362],
        ...,
        [ 1.0776],
        [-0.4885],
        [-0.4208]])

In [130]:
# let build a model
price_prediction_model = nn.Sequential(
    nn.Linear(in_features=7,out_features=10),
    nn.ReLU(),
    nn.Linear(in_features=10,out_features=10),
    nn.ReLU(),
    nn.Linear(in_features=10,out_features=10),
    nn.ReLU(),
    nn.Linear(in_features=10,out_features=10),
    nn.ReLU(),
    nn.Linear(in_features=10,out_features=10),
    nn.Dropout(p=0.2),
    nn.ReLU(),
    nn.Linear(in_features=10,out_features=10),
    nn.ReLU(),
    nn.Linear(in_features=10,out_features=1)
)
# let build the same version of the model but a class
class PricePredictionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=7,out_features=10)

        self.layer_2 = nn.Linear(in_features=10,out_features=10)
        self.layer_3 = nn.Linear(in_features=10,out_features=1)
    def forward(self,x):
        return self.layer_3(self.layer_2(self.layer_1(x)))
''' the two model are the same  '''

' the two model are the same  '

In [131]:
model = price_prediction_model().to(device)
model

TypeError: Sequential.forward() missing 1 required positional argument: 'input'

In [132]:
# lets make dammy predictions without teaching first the model
with torch.inference_mode():
    y_preds = model(labels.to(device))

print(y_preds)

tensor([[ 0.2798],
        [-0.2458],
        [ 0.3944],
        ...,
        [ 0.3796],
        [-0.6207],
        [-0.3096]], device='cuda:0')


In [133]:
# check the accuracy of the model
from sklearn.metrics import r2_score
r2_score(target,y_preds.cpu())

0.6928751473143703

In [134]:
#model(labels.to(device))
labels

tensor([[-0.2071,  1.0510, -0.7934,  ...,  0.7897,  0.1638,  0.9122],
        [-0.0919,  0.1567,  0.2571,  ...,  0.7897,  0.1638, -0.3187],
        [ 0.0735,  0.9848, -0.6278,  ...,  0.7897,  0.1638, -0.3187],
        ...,
        [-0.1478, -1.0025,  0.0657,  ...,  0.7897,  1.3900,  1.5277],
        [-0.0802, -0.7044, -0.2190,  ..., -1.0260, -1.0625, -0.9341],
        [-0.0581, -0.2076,  0.2416,  ..., -1.0260,  0.1638, -0.3187]])

In [135]:
''' my r2_score is negative meaning the model is performing worse than every so we have to train it  we can also use RMSE which will be used to evaluate our model'''
from sklearn.metrics import mean_squared_error
mean_squared_error(target,y_preds.cpu())

0.30712485

In [136]:
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.01)
loss_fn = nn.L1Loss()

epochs = 1000

for epoch in range(epochs):
    model.train()
    y_pred = model(labels.to(device))
    loss = loss_fn(y_pred,target.to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    with torch.inference_mode():
        test_pred = model(labels.to(device))
        test_loss = loss_fn(test_pred,target.to(device))
        if epoch % 100 == 0:
          print(f'Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss}')


Epoch: 0 | Loss: 0.33842965960502625 | Test Loss: 0.338429719209671
Epoch: 100 | Loss: 0.33842965960502625 | Test Loss: 0.3384295701980591
Epoch: 200 | Loss: 0.33842962980270386 | Test Loss: 0.33842965960502625
Epoch: 300 | Loss: 0.33842960000038147 | Test Loss: 0.33842965960502625
Epoch: 400 | Loss: 0.33842965960502625 | Test Loss: 0.33842962980270386
Epoch: 500 | Loss: 0.3384295701980591 | Test Loss: 0.33842962980270386
Epoch: 600 | Loss: 0.33842965960502625 | Test Loss: 0.33842968940734863
Epoch: 700 | Loss: 0.33842965960502625 | Test Loss: 0.3384297788143158
Epoch: 800 | Loss: 0.3384295701980591 | Test Loss: 0.3384297788143158
Epoch: 900 | Loss: 0.33842960000038147 | Test Loss: 0.3384298086166382


In [137]:
labels

tensor([[-0.2071,  1.0510, -0.7934,  ...,  0.7897,  0.1638,  0.9122],
        [-0.0919,  0.1567,  0.2571,  ...,  0.7897,  0.1638, -0.3187],
        [ 0.0735,  0.9848, -0.6278,  ...,  0.7897,  0.1638, -0.3187],
        ...,
        [-0.1478, -1.0025,  0.0657,  ...,  0.7897,  1.3900,  1.5277],
        [-0.0802, -0.7044, -0.2190,  ..., -1.0260, -1.0625, -0.9341],
        [-0.0581, -0.2076,  0.2416,  ..., -1.0260,  0.1638, -0.3187]])

In [138]:
with torch.inference_mode():
    y_preds = model(labels.to(device))


y_preds

tensor([[ 0.2798],
        [-0.2458],
        [ 0.3943],
        ...,
        [ 0.3797],
        [-0.6207],
        [-0.3096]], device='cuda:0')